# Introduzione a Redis

## Nozioni Base
Redis è un in-memory data structure store open source (BSD licensed)usato come database, cache, message broker e streaming engine. Redis dispone di data structures come strings, hashes, lists, sets, sorted sets con diverse queries, bitmaps, hyperloglogs, geospatial indexes, e streams. Redis ha un built-in replica system che permette di creare repliche del medesimo DB, Lua scripting, LRU eviction, transazioni e diversi livelli di on-disk persistence.

Per ottenere il massimo delle performance, Redis utilizza un dataset in-memory. In base al caso d'uso, Redis può salvare i dati dumpando periodicamente il dataset su disco, oppure aggiungere ad un log ogni operazione (comando) effettuato.

Redis può essere usato su quasi tutti i linguaggi di programmazione disponibili.

## Chi usa Redis?
Le compagnie più note che usano Redis sono:
- Twitter
- Github
- Snapchat
- Cragslist
- StackOverflow

E molte altre ancora.

## Installazione
### Windows
Redis non è supportato su Windows, ma si può installare sfruttando la WSL2 (Windows Subsystem for Linux).
Con una shell WSL correttamente inizializzata si potranno seguire gli step di installazione su macchina Ubuntu/Debian per installare l'ultima versione di Redis.

```bash
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis
```

E per avviare il servizio basterà lanciare il comando:
```bash
sudo service redis-server start
```

### Linux
Avendo in commercio una gran quantità di distribuzioni Linux, ogni package manager di riferimento ha una guida su come installare il pacchetto Redis.
Come spiegato nel capitolo per Windows, su sistemi Debian/Ubuntu possiamo aggiungere il Repository APT Ufficiale `packages.redis.io` e installare successivamente il pacchetto:
```bash
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis
```

Alternativamente possiamo sfruttare il pacchetto Redis disponibile sullo Snapcraft Store e installarlo su tutte le distribuzioni che lo supportano:

```bash
sudo snap install redis
```

### MacOS
Per installare il pacchetto Redis su dispositivi macOS, sarà necessario in prima battuta installare Brew, un package manager creato per il Sistema Operativo di Cupertino.
Una volta installato correttamente si dovrà semplicemente lanciare il comando:
```bash
brew install redis
```

Per avviare il servizio si dovrà usare:
```bash
redis-server
```

Per fermare il servizio andrà premuto `CTRL-C`

## Pro e Contro
PRO:

- Velocità: Redis è estremamente veloce, in grado di gestire grandi quantità di dati in modo rapido e efficiente, grazie alla sua architettura in memoria.

- Scalabilità: Redis è in grado di scalare orizzontalmente, ovvero di aggiungere nodi al cluster per aumentare la capacità di gestione dei dati.

- API: Redis offre una vasta gamma di API per supportare la gestione dei dati, tra cui la memorizzazione di dati semplici, liste, set e mappe.

- Persistenza dei dati: Redis offre la possibilità di persistere i dati su disco, garantendo la durata dei dati in caso di malfunzionamenti del server.

- Architettura semplice: Redis è facile da utilizzare e da configurare, anche per gli utenti meno esperti.

CONTRO:

- Scalabilità: Anche se Redis è in grado di scalare orizzontalmente, aggiungendo nodi ai cluster, non sfrutta al massimo le capacità della singola macchina. Per questo sono state create versioni multi-thread ad esempio: [cachegrand.io](https://cachegrand.io/)

- Capacità limitata: Poiché Redis utilizza la memoria per archiviare i dati, la capacità di archiviazione è limitata dalla quantità di memoria disponibile sul server.

- Nessun supporto per SQL: Redis non supporta SQL, quindi non è adatto per applicazioni che richiedono complesse operazioni di query.

- Persistenza dei dati: Anche se Redis offre la possibilità di persistere i dati su disco, questa funzionalità può rallentare le prestazioni del server.

- Nessun supporto per transazioni complesse: Redis non supporta le transazioni complesse, il che lo rende meno adatto per le applicazioni che richiedono operazioni di transazione complesse e transazioni ACID.

## Librerie del DBMS
Quasi tutti i linguaggi di programmazione hanno disponibile un porting per connettersi al servizio Redis, lavorando con Python andremo nel dettaglio di quest'ultima.
### redis-py
**Installazione**

Per installare redis-py:
```bash
pip install redis-py
```

Per migliorare le performance si può utilizzare installa il pacchetto con il supporto ad `hiredis`, che offre un Response Parser compilato, che per molte situazioni richiede 0 modifiche.

```bash
pip install redis[hiredis]
```

Link alla documentazione di [redis-py](https://redis-py.readthedocs.io/en/stable/)

Installazione di redis-py su Jupiter:

In [1]:
%pip install redis

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Creazione e Connessione al DB
Per facilitare lo sviluppo su più sistemi operativi, questo progetto dispone di un docker-compose.yml file, contenente il necessario per avviare un'istanza redis con due User Interfaces, quali `redis-cli` e `RedisInsight`.

Per avviare il servizio si può lanciare il comando:
```bash
make start
```

### Connessione al DB tramite redis-py

In [2]:
import redis
from redis.commands.json.path import Path
from redis.commands.search.field import TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

## Operazioni CRUD
In Redis, le operazioni CRUD (Create, Read, Update, Delete) sono eseguite tramite comandi specifici per la manipolazione dei dati.

Ecco una panoramica delle operazioni CRUD in Redis:

- CREATE: L'operazione di creazione di un nuovo dato in Redis viene eseguita utilizzando il comando SET. Ad esempio, per creare un nuovo valore "valore1" associato alla chiave "chiave1", si può utilizzare il comando SET chiave1 valore1.

In [3]:
r.set('nome', 'Marco')

True

- READ: Per leggere i dati associati a una chiave, si può utilizzare il comando GET. Ad esempio, per leggere il valore associato alla chiave "chiave1", si può utilizzare il comando GET chiave1.

In [4]:
r.get('nome')

'Marco'

- UPDATE: L'operazione di aggiornamento di un dato esistente in Redis viene eseguita utilizzando il comando SET. Ad esempio, per aggiornare il valore associato alla chiave "chiave1" con il nuovo valore "valore2", si può utilizzare il comando SET chiave1 valore2.

In [5]:
r.set('nome', 'Francesco')

True


- DELETE: Per eliminare una chiave e il valore associato, si può utilizzare il comando DEL. Ad esempio, per eliminare la chiave "chiave1" e il valore associato ad essa, si può utilizzare il comando DEL chiave1.

In [6]:
r.delete('nome')

1

Inoltre, Redis offre anche altri comandi per manipolare i dati in modo più specifico, come ad esempio il comando INCR per incrementare il valore numerico di una chiave, il comando LPUSH per aggiungere un valore a una lista, e il comando SADD per aggiungere un elemento a un set.

## Comparazione con i DBMS visti a lezione

Redis è un DB NoSQL che offre velocità, affidabilità e prestazioni, posto a confronto con altri DB abbiamo alcune differenze:

- Nel DB in analisi la differenza principale nel salvataggio dei dati in memoria è che questi non vengono salvati nella memoria secondaria ma bensì nella memoria primaria rendendo le operazioni di lettura e di scrittura molto veloci 
- La struttura delle query è molto semplice e varia in base alla versione di Redis che si utilizza: bisogna definire la funzione da eseguire e successivamente i parametri.

In [7]:
r.rpush("mylist", 1, 2, 3, 4, 5, "hello world")
# RPUSH mylist 1 2 3 4 5 "hello world"

24

&emsp;&emsp;&ensp;La funzione sopra indicata permette di eseguire una push in una lista aggiungendo sei elementi, per poter vedere gli elementi all'interno di una lista basta semplicemente eseguire:

In [8]:
r.lrange("mylist", 0, -1)

['1',
 '2',
 '3',
 '4',
 '5',
 'hello world',
 '1',
 '2',
 '3',
 '4',
 '5',
 'hello world',
 '1',
 '2',
 '3',
 '4',
 '5',
 'hello world',
 '1',
 '2',
 '3',
 '4',
 '5',
 'hello world']

Da questi esempi (non complessi) si può notare la semplicità e la facilità di uso di questo DB. 
In poco tempo con il supporto della documentazione si può imparare e si può utilizzare al meglio questo strumento.
Infine la documentazione presente sul sito di <link href="redis.io">redis.io</link> è disponibile solamente in inglese, permette di seguire l'utente dall'installazione all'utilizzo completo del Database e consente di imparare tutte le nozioni base (e anche non) in poco tempo in una guida descritta semplicemente e passo passo. 

## Esempi di query (Complesse)
E' importante notare che Redis non è un database relazionale tradizionale con un linguaggio di query completo come SQL. Redis è un database di strutture dati in memoria che offre un set di comandi per la manipolazione di essi.

Nel contesto di Redis, le query complesse sono solitamente realizzate concatenando i comandi disponibili nel Redis Command Language. Vengono forniti un insieme di comandi predefiniti per interagire con Redis, come SET, GET, HSET, HGET, ecc., che possono essere utilizzati per leggere, scrivere o manipolare i dati all'interno delle strutture di dati come stringhe, liste, set, hash, ecc.

Redis fornisce inoltre un sistema di scripting integrato, tramite il linguaggio Lua, che permette di eseguire più comandi e creare logiche complesse.

Ai fini del progetto verranno introdotto due moduli esterni, ma disponibili all'interno della docker image <code>redis-stack</code> denominati [RediSearch](https://redis.io/docs/stack/search) e [RedisJSON](https://redis.io/docs/stack/json/), redis-stack contiene altri moduli per il supporto a Grafi e TimeSeries.

## Introduzione ai Moduli Esterni
RediSearch è un modulo open source di Redis che abilita all'uso di query, secondary indexing e full-text search sul DB. Queste funzionalità permetto queries su più fields, aggregation, exact phrase matching, come visto anche all'interno di ElasticSearch.
Tramite questo modulo possiamo dunque simulare all'interno di redis un comportamento simile a quello di ES.

Il modulo RedisJSON abilita al supporto per i JSON, permettendo di salvare, aggiornare e recuperare dei JSON values all'interno del DB, in modo similare ai tipi già coperti da Redis. RedisJSON funziona autoamaticamente con RediSearch permettendo di creare indici e effettuare query su documenti JSON.

In [9]:
## Creiamo lo schema per l'indice
schema = (NumericField('$.id', as_name='id'),
          TextField('$.name.english', as_name='english'),
          TextField('$.name.japanese', as_name='japanese'),
          TextField('$.name.chinese', as_name='chinese'),
          TextField('$.name.french', as_name='french'),
          TextField('$.type', as_name='type'),
          NumericField('$.base.HP', as_name='hp'),
          NumericField('$.base.Attack', as_name='att'),
          NumericField('$.base.Defense', as_name='def'),
          NumericField('$.base.Sp. Attack', as_name='spatt'),
          NumericField('$.base.Sp. Defense', as_name='spdef'),
          NumericField('$.base.Speed', as_name='spd'))

## Creiamo l'indice se non esiste
try:
    r.ft("pokemon").create_index(schema, definition=IndexDefinition(
        prefix=['pokemon:'], index_type=IndexType.JSON))
except:
    # passa oltre se esiste già
    pass

from redis.commands.search.query import NumericFilter, Query
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers

- Cerca tutti i pokemon che contengono Pika nel loro nome:

In [10]:
r.ft("pokemon").search("Pika*")

Result{1 total, docs: [Document {'id': 'pokemon:25', 'payload': None, 'json': '{"id":25,"name":{"english":"Pikachu","japanese":"ピカチュウ","chinese":"皮卡丘","french":"Pikachu"},"type":["Electric"],"base":{"HP":35,"Attack":55,"Defense":40,"Sp. Attack":50,"Sp. Defense":50,"Speed":90}}'}]}

Tempo di Esecuzione: 4.778 ms

- Cerca tutti i pokemon di tipo Fuoco (contenendo anche i tipi misti come Fuoco-Volante)

In [11]:
query = Query("@type:Fire")
r.ft("pokemon").search(query)

Result{64 total, docs: [Document {'id': 'pokemon:255', 'payload': None, 'json': '{"id":255,"name":{"english":"Torchic","japanese":"アチャモ","chinese":"火稚鸡","french":"Poussifeu"},"type":["Fire"],"base":{"HP":45,"Attack":60,"Defense":40,"Sp. Attack":70,"Sp. Defense":50,"Speed":45}}'}, Document {'id': 'pokemon:498', 'payload': None, 'json': '{"id":498,"name":{"english":"Tepig","japanese":"ポカブ","chinese":"暖暖猪","french":"Gruikui"},"type":["Fire"],"base":{"HP":65,"Attack":63,"Defense":45,"Sp. Attack":45,"Sp. Defense":45,"Speed":45}}'}, Document {'id': 'pokemon:725', 'payload': None, 'json': '{"id":725,"name":{"english":"Litten","japanese":"ニャビー","chinese":"火斑喵","french":"Flamiaou"},"type":["Fire"],"base":{"HP":45,"Attack":65,"Defense":40,"Sp. Attack":60,"Sp. Defense":40,"Speed":70}}'}, Document {'id': 'pokemon:257', 'payload': None, 'json': '{"id":257,"name":{"english":"Blaziken","japanese":"バシャーモ","chinese":"火焰鸡","french":"Braségali"},"type":["Fire","Fighting"],"base":{"HP":80,"Attack":120,"De

Tempo di Esecuzione: 6.353 ms

- Cerca i primi 10 pokemon che hanno un attacco compreso fra 50 e 100 (estremi inclusi)

In [12]:
query = Query("@att:[50 100]").return_fields("english", "att").paging(0, 10)
r.ft("pokemon").search(query).docs

[Document {'id': 'pokemon:82', 'payload': None, 'english': 'Magneton', 'att': '60'},
 Document {'id': 'pokemon:518', 'payload': None, 'english': 'Musharna', 'att': '55'},
 Document {'id': 'pokemon:159', 'payload': None, 'english': 'Croconaw', 'att': '80'},
 Document {'id': 'pokemon:387', 'payload': None, 'english': 'Turtwig', 'att': '68'},
 Document {'id': 'pokemon:305', 'payload': None, 'english': 'Lairon', 'att': '90'},
 Document {'id': 'pokemon:358', 'payload': None, 'english': 'Chimecho', 'att': '50'},
 Document {'id': 'pokemon:158', 'payload': None, 'english': 'Totodile', 'att': '65'},
 Document {'id': 'pokemon:501', 'payload': None, 'english': 'Oshawott', 'att': '55'},
 Document {'id': 'pokemon:460', 'payload': None, 'english': 'Abomasnow', 'att': '92'},
 Document {'id': 'pokemon:255', 'payload': None, 'english': 'Torchic', 'att': '60'}]

Tempo di Esecuzione: 2.457 ms

- Trovare tutti i pokémon con difesa minore di 90 o di tipo “Flying”. Proiettare solo l’attacco, il nome in inglese e i tipi. Limitare i risultati da visualizzare a 10.

In [13]:
query = Query("@def:[0 90] | @type:Fire").return_fields("english", "type", "att").paging(0, 10)
r.ft("pokemon").search(query).docs

[Document {'id': 'pokemon:255', 'payload': None, 'english': 'Torchic', 'type': '["Fire"]', 'att': '60'},
 Document {'id': 'pokemon:498', 'payload': None, 'english': 'Tepig', 'type': '["Fire"]', 'att': '63'},
 Document {'id': 'pokemon:725', 'payload': None, 'english': 'Litten', 'type': '["Fire"]', 'att': '65'},
 Document {'id': 'pokemon:257', 'payload': None, 'english': 'Blaziken', 'type': '["Fire","Fighting"]', 'att': '120'},
 Document {'id': 'pokemon:229', 'payload': None, 'english': 'Houndoom', 'type': '["Dark","Fire"]', 'att': '90'},
 Document {'id': 'pokemon:667', 'payload': None, 'english': 'Litleo', 'type': '["Fire","Normal"]', 'att': '50'},
 Document {'id': 'pokemon:637', 'payload': None, 'english': 'Volcarona', 'type': '["Bug","Fire"]', 'att': '60'},
 Document {'id': 'pokemon:608', 'payload': None, 'english': 'Lampent', 'type': '["Ghost","Fire"]', 'att': '40'},
 Document {'id': 'pokemon:513', 'payload': None, 'english': 'Pansear', 'type': '["Fire"]', 'att': '53'},
 Document {'i

Tempo di Esecuzione: 3.846 ms

- Trovare la media dei valori di Attacco per tutti i tipi

In [14]:
req = aggregations.AggregateRequest().group_by("@type", reducers.avg("@att"))
r.ft("pokemon").aggregate(req).rows

[['type', '["Electric","Ghost"]', '__generated_aliasavgatt', '50'],
 ['type', '["Bug","Poison"]', '__generated_aliasavgatt', '60.9090909091'],
 ['type', '["Electric","Flying"]', '__generated_aliasavgatt', '93.3333333333'],
 ['type', '["Normal","Flying"]', '__generated_aliasavgatt', '76.0769230769'],
 ['type', '["Fire"]', '__generated_aliasavgatt', '78.2'],
 ['type', '["Fighting","Psychic"]', '__generated_aliasavgatt', '50'],
 ['type', '["Fighting","Ghost"]', '__generated_aliasavgatt', '125'],
 ['type', '["Grass","Ground"]', '__generated_aliasavgatt', '109'],
 ['type', '["Poison","Fighting"]', '__generated_aliasavgatt', '83.5'],
 ['type', '["Psychic","Fire"]', '__generated_aliasavgatt', '100'],
 ['type', '["Water","Ground"]', '__generated_aliasavgatt', '77.1111111111'],
 ['type', '["Grass","Dark"]', '__generated_aliasavgatt', '95'],
 ['type', '["Ice","Water"]', '__generated_aliasavgatt', '60'],
 ['type', '["Steel","Psychic"]', '__generated_aliasavgatt', '79.6666666667'],
 ['type', '["Fi

Tempo di Esecuzione: 26.126 ms

### Analisi delle tempistiche delle Query

# Contributors
Lorenzo Emanuele Avallone 866163\
Mattia Signorelli 852347